# Time Series Analysis - Figures for Slides

This notebook generates figures for the Time Series lecture slides.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import json

# Set style for dark slides
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16

## 1. Fourier Decomposition Example

In [ ]:
# Generate a composite signal
t = np.linspace(0, 2, 500)
f1, f2, f3 = 2, 5, 8  # frequencies
signal = 1.0*np.sin(2*np.pi*f1*t) + 0.5*np.sin(2*np.pi*f2*t) + 0.3*np.sin(2*np.pi*f3*t)

fig, axes = plt.subplots(2, 1, figsize=(12, 6))

# Time domain
axes[0].plot(t, signal, color='#61afef', linewidth=1.5)
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Composite Signal: $\sin(2\pi \cdot 2t) + 0.5\sin(2\pi \cdot 5t) + 0.3\sin(2\pi \cdot 8t)$')
axes[0].grid(alpha=0.3)

# Frequency domain (FFT)
N = len(t)
dt = t[1] - t[0]
freqs = fftfreq(N, dt)[:N//2]
fft_vals = np.abs(fft(signal))[:N//2] * 2/N

axes[1].stem(freqs[:50], fft_vals[:50], linefmt='#98c379', markerfmt='o', basefmt=' ')
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Amplitude')
axes[1].set_title('Fourier Transform (Frequency Domain)')
axes[1].set_xlim(0, 12)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('fourier_decomposition.png', dpi=150, bbox_inches='tight', facecolor='#282c34')
plt.show()

## 2. AR(2) Model Fitting a Sine Wave

In [ ]:
# Generate sine wave data
N = 200
omega = 0.3  # radians per step
i_vals = np.arange(N)
x_vals = np.sin(omega * i_vals)

# Create lagged data for AR(2)
X = np.column_stack([x_vals[1:-1], x_vals[:-2], np.ones(N-2)])
y = x_vals[2:]

# Fit AR(2) via least squares
w_hat = np.linalg.lstsq(X, y, rcond=None)[0]
w1, w2, w0 = w_hat

print(f"Fitted AR(2) parameters:")
print(f"  w1 = {w1:.4f} (theory: {2*np.cos(omega):.4f})")
print(f"  w2 = {w2:.4f} (theory: -1.0)")
print(f"  w0 = {w0:.4f} (theory: 0.0)")
print(f"\nRecovered omega = {np.arccos(w1/2):.4f} (true: {omega:.4f})")

# Predict using AR(2)
x_pred = np.zeros(N)
x_pred[:2] = x_vals[:2]
for i in range(2, N):
    x_pred[i] = w1*x_pred[i-1] + w2*x_pred[i-2] + w0

# Plot
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(i_vals, x_vals, color='#61afef', linewidth=2, label='True: $\sin(\omega i)$')
ax.plot(i_vals, x_pred, '--', color='#e06c75', linewidth=2, alpha=0.8, label='AR(2) Prediction')
ax.set_xlabel('Time step $i$')
ax.set_ylabel('$x_i$')
ax.set_title(f'AR(2) Captures Sine Wave: $\omega = {omega}$')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('ar2_sine.png', dpi=150, bbox_inches='tight', facecolor='#282c34')
plt.show()

## 3. TCN Dilated Convolution Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Draw TCN layers with dilations
layers = ['Input', 'Conv d=1', 'Conv d=2', 'Conv d=4', 'Output']
n_timesteps = 16
colors = ['#abb2bf', '#61afef', '#98c379', '#e5c07b', '#c678dd']

for layer_idx, (layer_name, color) in enumerate(zip(layers, colors)):
    y = layer_idx * 1.5
    for t in range(n_timesteps):
        circle = plt.Circle((t, y), 0.3, color=color, alpha=0.8)
        ax.add_patch(circle)
    ax.text(-1.5, y, layer_name, fontsize=11, va='center', ha='right', color='white')

# Draw connections showing dilation pattern
dilations = [1, 2, 4]
for layer_idx, d in enumerate(dilations):
    y_from = layer_idx * 1.5
    y_to = (layer_idx + 1) * 1.5
    # Show connections for t=15 (rightmost output)
    t_out = 15
    for k in range(3):  # kernel size 3
        t_in = t_out - k * d
        if t_in >= 0:
            ax.plot([t_in, t_out], [y_from + 0.3, y_to - 0.3], 
                   color='#e06c75', alpha=0.6, linewidth=1.5)

ax.set_xlim(-3, 17)
ax.set_ylim(-1, 7)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('TCN: Dilated Causal Convolutions (kernel=3)', fontsize=14, color='white')

plt.tight_layout()
plt.savefig('tcn_architecture.png', dpi=150, bbox_inches='tight', facecolor='#282c34')
plt.show()

## 4. State-Space / Kalman Filter Illustration

In [ ]:
# Simple 1D tracking with Kalman filter
np.random.seed(42)
T = 50

# True state (position following a smooth trajectory)
true_state = np.cumsum(np.sin(np.linspace(0, 4*np.pi, T)) * 0.5)

# Noisy observations
obs_noise = 1.5
observations = true_state + np.random.randn(T) * obs_noise

# Simple Kalman filter
A = 1.0  # State transition
C = 1.0  # Observation
Q = 0.1  # Process noise
R = obs_noise**2  # Measurement noise

x_est = np.zeros(T)
P = 1.0  # Initial covariance
x = 0.0  # Initial estimate

for i in range(T):
    # Predict
    x_pred = A * x
    P_pred = A * P * A + Q
    
    # Update
    K = P_pred * C / (C * P_pred * C + R)  # Kalman gain
    x = x_pred + K * (observations[i] - C * x_pred)
    P = (1 - K * C) * P_pred
    
    x_est[i] = x

# Plot
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(true_state, color='#98c379', linewidth=2, label='True State')
ax.scatter(range(T), observations, color='#e06c75', s=30, alpha=0.6, label='Noisy Observations')
ax.plot(x_est, color='#61afef', linewidth=2, label='Kalman Filter Estimate')
ax.set_xlabel('Time step')
ax.set_ylabel('Position')
ax.set_title('Kalman Filter: Estimating Hidden State from Noisy Observations')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('kalman_filter.png', dpi=150, bbox_inches='tight', facecolor='#282c34')
plt.show()

## 5. Export Data for D3.js Interactive Plots

In [ ]:
# Export data for D3.js visualizations
export_data = {
    'sine_wave': {
        't': list(np.linspace(0, 4*np.pi, 200)),
        'default_omega': 1.0
    },
    'ar2_demo': {
        'i_vals': list(range(100)),
        'default_omega': 0.3
    },
    'fourier_components': [
        {'freq': 2, 'amp': 1.0},
        {'freq': 5, 'amp': 0.5},
        {'freq': 8, 'amp': 0.3}
    ]
}

with open('slide_data.json', 'w') as f:
    json.dump(export_data, f, indent=2)

print("Data exported to slide_data.json")